Latest update: 2022-11-14 10:48:25.184356

<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/koster_data_management/blob/main/tutorials/03_Upload_clips_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso_data_management and its requirements</font> { vertical-output: true }

try:
    import google.colab
    import os
    from IPython.display import clear_output

    IN_COLAB = True
    print("Running in Colab...")

    # Clone koster_data_management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/koster_data_management.git
    !pip install -q --upgrade pip
    !pip install -q -r koster_data_management/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("koster_data_management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys

    if "ipysheet" not in sys.modules:
        !pip install -q --no-warn-script-location --upgrade pip
        !pip install -q ipysheet==0.4.4

    print("Running locally... you're good to go!")

#######Import Python packages########

# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.movie_utils as m_utils
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils

print("Packages loaded successfully")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify GPU availability</font> { vertical-output: true }
gpu_available = t_utils.gpu_select()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }
project_name = t_utils.choose_project()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Initiate project's database</font> { vertical-output: true }
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)

# Initiate db
db_info_dict = t_utils.initiate_db(project)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Connect to Zooniverse</font> { vertical-output: true }
zoo_project = t_utils.connect_zoo_project(project)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Retrieve information from Zooniverse and media storage</font> { vertical-output: true }

# Retrieve info about zooniverse clips
zoo_info_dict = t_utils.retrieve__populate_zoo_info(
    project=project,
    db_info_dict=db_info_dict,
    zoo_project=zoo_project,
    zoo_info=["subjects"],
)

# Retrieve info about the movies available in the server
available_movies_df = s_utils.retrieve_movie_info_from_server(
    project=project, db_info_dict=db_info_dict
)

# Select the movie to upload to Zooniverse

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify movie of interest</font> { vertical-output: true }
movie_selected = t_utils.select_movie(available_movies_df)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Preview the selected movie</font> { vertical-output: true }
movie_display, movie_path = t_utils.preview_movie(
    project=project,
    db_info_dict=db_info_dict,
    available_movies_df=available_movies_df,
    movie_i=movie_selected.value,
)
movie_display

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check if movie is already in Zooniverse</font> { vertical-output: true }

# Remember to query the newest zooniverse data to get the most up to date list of clips uploaded
t3.check_movie_uploaded(movie_i=movie_selected.value, db_info_dict=db_info_dict)

# Create some clip examples (Optional)
Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select how many clip examples</font> { vertical-output: true }
random_clip_selection = t3.select_random_clips(
    movie_i=movie_selected.value, db_info_dict=db_info_dict
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Create the clip examples</font> { vertical-output: true }
example_clips = t3.create_example_clips(
    movie_i=movie_selected.value,
    movie_path=movie_path,
    db_info_dict=db_info_dict,
    project=project,
    clip_selection=random_clip_selection,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Review the size of the clips</font> { vertical-output: true }
t3.check_clip_size(clips_list=example_clips)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select modifications for the clips</font> { vertical-output: true }

clip_example_modification = t3.clip_modification_widget()
clip_example_modification

In [ ]:
# @title <font size="5">↓ ឵឵<i>Modify the clips</font> { vertical-output: true }
modified_clips = t3.create_modified_clips(
    clips_list=example_clips,
    movie_i=movie_selected.value,
    modification_details=clip_example_modification.checks,
    project=project,
    gpu_available=gpu_available.result,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check size of the modified clips</font> { vertical-output: true }
t3.check_clip_size(clips_list=modified_clips)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Display clips side-by-side (in progress)</font> { vertical-output: true }
t3.compare_clips(example_clips=example_clips, modified_clips=modified_clips)

# Create the clips to upload to Zooniverse

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select video modifications for the clips</font> { vertical-output: true }

clip_modification = t3.clip_modification_widget()
clip_modification

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify the number of clips and clip length</font> { vertical-output: true }
clip_selection = t3.select_clip_n_len(
    movie_i=movie_selected.value, db_info_dict=db_info_dict
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check information about the clips</font> { vertical-output: true }
t3.review_clip_selection(
    clip_selection=clip_selection,
    movie_i=movie_selected.value,
    clip_modification=clip_modification,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Create the clips</font> { vertical-output: true }

clips_to_upload_df = t3.create_clips(
    available_movies_df=available_movies_df,
    movie_i=movie_selected.value,
    movie_path=movie_path,
    db_info_dict=db_info_dict,
    clip_selection=clip_selection,
    project=project,
    modification_details=clip_modification.checks,
    gpu_available=gpu_available.result,
    pool_size=10,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check the size of the clips</font> { vertical-output: true }

t3.check_clip_size(clips_list=clips_to_upload_df.clip_path.to_list())

# Upload clips to Zooniverse
Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
# @title <font size="5">↓ ឵឵<i>Set Zooniverse metadata</font> { vertical-output: true }
upload_to_zoo, sitename, created_on = t3.set_zoo_metadata(
    db_info_dict=db_info_dict, df=clips_to_upload_df, project=project
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Upload clips to Zooniverse</font> { vertical-output: true }

t3.upload_clips_to_zooniverse(
    upload_to_zoo=upload_to_zoo,
    sitename=sitename,
    created_on=created_on,
    project=project.Zooniverse_number,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Remove local clips to free up space for new clips</font> { vertical-output: true }

t3.remove_temp_clips(upload_to_zoo)

In [ ]:
# END